In [1]:
import pydantic
print(pydantic.VERSION)


2.5.3


Custom Validators 
- Customizing Field Validators

In [2]:
from pydantic import BaseModel, Field, ValidationError

class User(BaseModel):
    age : int = Field(gt=31)
    name : str

try:
    u1 = User(age=12, name="mona")
    print(u1)
except pydantic.ValidationError as e:
    print(e)

1 validation error for User
age
  Input should be greater than 31 [type=greater_than, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/greater_than


In [3]:
from pydantic import BaseModel, Field, ValidationError
from pydantic import field_validator

class User(BaseModel):
    age : int = Field(gt=0)
    name : str

    @field_validator("age")
    @classmethod
    def validate_age(cls, value):
        if value < 18:
            raise ValueError("Age must be atleast 18")
        return value

In [4]:
try:
    u1 = User(age=11, name="kiko")
    print(u1)
except pydantic.ValidationError as e:
    print(e)

1 validation error for User
age
  Value error, Age must be atleast 18 [type=value_error, input_value=11, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


In [6]:
try:
    u2 = User(age=18, name="kiko")
    print(u2)
except pydantic.ValidationError as e:
    print(e)

age=18 name='kiko'


In [11]:
from datetime import date
from pydantic import BaseModel, ValidationError


class ScheduledCourse(BaseModel):
    dept : str
    course_name : str
    start_dt : date
    end_dt : date


try:
    c1 = ScheduledCourse(
        dept="Alumina",
        course_name="Science",
        start_dt="2025-12-22",
        end_dt="2024-11-12"
    )
    print(c1)
except pydantic.ValidationError as e:
    print(e)

dept='Alumina' course_name='Science' start_dt=datetime.date(2025, 12, 22) end_dt=datetime.date(2024, 11, 12)


Model Level Validators : 

ex. > If I want start_date to be always before end_date


In [15]:
from pydantic import BaseModel, ValidationError, model_validator

class ScheduledCourse(BaseModel):
    dept : str
    course_name : str
    start_dt : date
    end_dt : date

    @model_validator(mode="before") # Using before the model is being instantiated
    @classmethod
    def validate_date(cls, data: dict):
        if data["start_dt"] > data["end_dt"]:
            raise ValueError("Start date must be before the End date")
        return data 

try:
    c1 = ScheduledCourse(
        dept="Alumina",
        course_name="Science",
        start_dt="2025-12-22",
        end_dt="2024-11-12"
    )
    print(c1)
except pydantic.ValidationError as e:
    print(e)



1 validation error for ScheduledCourse
  Value error, Start date must be before the End date [type=value_error, input_value={'dept': 'Alumina', 'cour... 'end_dt': '2024-11-12'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


In [17]:
try:
    c2 = ScheduledCourse(
        dept="Alumina",
        course_name="Science",
        start_dt="2024-11-22",
        end_dt="2024-12-12"
    )
    print(c2)
except pydantic.ValidationError as e:
    print(e)

dept='Alumina' course_name='Science' start_dt=datetime.date(2024, 11, 22) end_dt=datetime.date(2024, 12, 12)


In [22]:
from pydantic import BaseModel, ValidationError, model_validator

class ScheduledCourse(BaseModel):
    dept : str
    course_name : str
    start_dt : date
    end_dt : date

    @model_validator(mode="after") # Using after the model is being instantiated
    def validate_date(self):
        if self.start_dt > self.end_dt:
            raise ValueError("Start date must be before the End date")
        return self 

try:
    c1 = ScheduledCourse(
        dept="Alumina",
        course_name="Science",
        start_dt="2025-12-22",
        end_dt="2024-11-12"
    )
    print(c1)
except pydantic.ValidationError as e:
    error = e
    



In [24]:
print(error)

1 validation error for ScheduledCourse
  Value error, Start date must be before the End date [type=value_error, input_value={'dept': 'Alumina', 'cour... 'end_dt': '2024-11-12'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


In [27]:
print(error.json())

[{"type":"value_error","loc":[],"msg":"Value error, Start date must be before the End date","input":{"dept":"Alumina","course_name":"Science","start_dt":"2025-12-22","end_dt":"2024-11-12"},"ctx":{"error":"Start date must be before the End date"},"url":"https://errors.pydantic.dev/2.5/v/value_error"}]


In [32]:
print(error.json(indent=2))

[
  {
    "type": "value_error",
    "loc": [],
    "msg": "Value error, Start date must be before the End date",
    "input": {
      "dept": "Alumina",
      "course_name": "Science",
      "start_dt": "2025-12-22",
      "end_dt": "2024-11-12"
    },
    "ctx": {
      "error": "Start date must be before the End date"
    },
    "url": "https://errors.pydantic.dev/2.5/v/value_error"
  }
]


In [34]:
print(error.errors()[0]["type"])
print(error.errors()[0]["msg"])

value_error
Value error, Start date must be before the End date


In [19]:
try:
    c2 = ScheduledCourse(
        dept="Alumina",
        course_name="Science",
        start_dt="2024-11-22",
        end_dt="2024-12-12"
    )
    print(c2)
except pydantic.ValidationError as e:
    print(e)

dept='Alumina' course_name='Science' start_dt=datetime.date(2024, 11, 22) end_dt=datetime.date(2024, 12, 12)
